In [12]:
from PIL import Image, ImageDraw
import random

def generate_image():
    # Create an image with a transparent background
    image = Image.new('RGBA', (512, 512), (0, 0, 0, 0))
    draw = ImageDraw.Draw(image)

    # Number of dots
    num_dots = 1600

    # Size of each dot
    dot_size = 3

    # Grid spacing
    grid_spacing = 12

    rand_radius = 2
    
    # Draw dots at grid centers with slight perturbations
    for x in range(0, 512, grid_spacing):
        for y in range(0, 512, grid_spacing):
            perturbation_x = random.randint(-rand_radius, rand_radius)
            perturbation_y = random.randint(-rand_radius, rand_radius)

            # Calculate dot position with perturbation
            dot_x = x + perturbation_x
            dot_y = y + perturbation_y

            # Draw the dot
            draw.ellipse((dot_x - dot_size, dot_y - dot_size, dot_x + dot_size, dot_y + dot_size), fill=(255, 255, 255, 255))

    # Save the image
    image.save('perturbed_dots.png')

if __name__ == "__main__":
    generate_image()

In [2]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt

def generate_perturbed_grid_points(grid_size, perturbation_range):
    points = np.array([[x, y] for x in range(1, grid_size) for y in range(1, grid_size)], dtype=float)
    points += np.random.uniform(-perturbation_range, perturbation_range, points.shape)
    return points * 8

def plot_colored_voronoi_diagram(points, grid_points):
    vor = Voronoi(points)
    voronoi_plot_2d(vor, show_vertices=False, line_colors='none', line_width=0, line_alpha=0, point_size=0)  # Set point_size to 0

    for i, region in enumerate(vor.regions[:-1]):
        if -1 not in region:
            polygon = [vor.vertices[i] for i in region]
            
            if (np.where(vor.point_region == i)[0].size > 0):
                point_id = np.where(vor.point_region == i)[0][0]
                grid_point = grid_points[point_id]
                region_color = np.array([grid_point[0]/512, grid_point[1]/512, 0])
                plt.fill(*zip(*polygon), color=region_color, edgecolor='none', alpha=1.0)
    #plt.show()
    plt.axis('off')
    plt.xlim(0, 512)
    plt.ylim(0, 512)
    
    fig = plt.gcf()

    # Set the figure size to ensure a 512x512 array
    fig.set_size_inches(8, 8)
    fig.set_dpi(512/8)
    
    fig.savefig('my_plot.png', bbox_inches='tight')
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    
    # Convert the Matplotlib plot to a NumPy array
    fig.canvas.draw()
    plot_array = np.array(fig.canvas.renderer.buffer_rgba())

    #plot_array = cv2.imread("my_plot.png")

    return plot_array

if __name__ == "__main__":
    grid_size = 64
    perturbation_range = 0.5

    # Generate perturbed grid points
    grid_points = generate_perturbed_grid_points(grid_size, perturbation_range)

    # Get colors for Voronoi cells
    vor = Voronoi(grid_points)
    #cell_colors = color_voronoi_cells(vor, grid_points)

    # Plot the colored Voronoi diagram with invisible edges and dots
    plot_array = plot_colored_voronoi_diagram(grid_points, grid_points)

In [105]:
from PIL import Image
im = Image.fromarray(plot_array)
im.save("voronoi.png")


In [3]:
import cv2
import numpy as np

img = cv2.imread('./Perlin_noise_example.png', 0).astype(float)
dirctn_field = np.zeros((img.shape[0], img.shape[1], 2), dtype=float)

for row in range(img.shape[0]):
    for col in range(img.shape[1]):
        if row == 0 or row == img.shape[0] - 1 or col == 0 or col == img.shape[1] - 1:
            dirctn_field[row][col] = np.zeros(2)
        else:
            dirctn = np.array([img[row][col+1] - img[row][col-1], img[row-1][col] - img[row+1][col]])
            if np.linalg.norm(dirctn) != 0:
                dirctn = dirctn*(1/np.linalg.norm(dirctn))
                dirctn_field[row][col] = dirctn
            else:
                dirctn_field[row][col] = np.zeros(2)
    
dirctn_field = dirctn_field[4:516, 4:516]

In [12]:
first_channel_image = (dirctn_field[:, :, 1] + 1) / 2

scaled_image = (first_channel_image * 255).astype(np.uint8)

# Display the 2-channel image with zeros in the third channel
cv2.imshow('Two-Channel Image', scaled_image)

# Wait for a key event and then close the window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
scaled_image

array([[ 70,  70,  37, ...,  13,  37,  37],
       [ 70,  50,  37, ...,  25,  25,  37],
       [ 87,  37,  50, ...,  18,  37,  37],
       ...,
       [248, 248, 233, ..., 158, 152, 158],
       [248, 217, 248, ..., 152, 148, 152],
       [217, 233, 217, ..., 145, 148, 148]], dtype=uint8)

In [18]:
from PIL import Image, ImageDraw
import random

def generate_random_between(low, high, sigma=0.2):
    random_num = random.gauss((low + high) / 2, sigma)
    return max(min(random_num, high), low)

def generate_images():
    # Number of images
    num_images = 15
    
    rand_radius = 2
    
    max_dot_size = 5
    min_dot_size = 1
    
    grid_spacing = 10

    # Create a list to store dot positions
    dot_positions = []
    
    dot_heights = []

    # Generate random dot positions        
    for x in range(0, 512, grid_spacing):
        for y in range(0, 512, grid_spacing):
            perturbation_x = random.randint(-rand_radius, rand_radius)
            perturbation_y = random.randint(-rand_radius, rand_radius)
            
            dot_positions.append((max(min(x + perturbation_x, 511), 0), max(min(y + perturbation_y, 511), 0)))
            dot_heights.append(generate_random_between(0.5, 1.0))

    images = {}
    draws = {}
    # Create images
    for img_num in range(num_images):
        # Create an image with a transparent background
        images[img_num] = Image.new('RGBA', (512, 512), (0, 0, 0, 0))
        draws[img_num] = ImageDraw.Draw(images[img_num])
            
    cluster_step_size = 1
    noise_step_size = 3

    # Draw dots at the same locations with progressively smaller sizes
    for index in range(len(dot_positions)):
        dot_loc = np.array(dot_positions[index])
        dot_height = dot_heights[index]
        grid_point = plot_array[511-dot_loc[1]][dot_loc[0]][:2] * (512/255)
        
        if grid_point[0] == 512 and grid_point[1] == 512:
            point_dir = np.array([0, 0])
        else:
#           noise_dir = dirctn_field[511-dot_loc[1]][dot_loc[0]]
            noise_dir = dirctn_field[int(511-grid_point[1])][int(grid_point[0])]        
            shifted_grid_point = grid_point + noise_step_size * noise_dir

            point_dir = (shifted_grid_point - dot_loc) / np.linalg.norm(shifted_grid_point - dot_loc)
        
        for img_num in range(num_images):
            alpha = img_num / (num_images * dot_height - 1)
            dot_size = max_dot_size * (1 - alpha) + min_dot_size * alpha  # Decrease dot size progressively
#             dot_loc_displaced = dot_loc + point_dir * cluster_step_size * img_num + noise_dir * noise_step_size * img_num
            dot_loc_displaced = dot_loc + point_dir * cluster_step_size * img_num
            dot_x, dot_y = dot_loc_displaced
            # Draw the dot
            draws[img_num].ellipse((dot_x - dot_size, dot_y - dot_size, dot_x + dot_size, dot_y + dot_size), fill=(255, 255, 255, 255))

    # Save the images
    for img_num in range(num_images):
        images[img_num].save(f'dots_{img_num + 1}.png')

if __name__ == "__main__":
    generate_images()

In [31]:
plot_array.shape


(399, 409, 3)

In [130]:
dirctn_field

array([[[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [-0.19611614, -0.98058068],
        [-0.31622777, -0.9486833 ],
        ...,
        [ 0.19611614, -0.98058068],
        [ 0.        , -1.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        , -1.        ],
        [-0.19611614, -0.98058068],
        ...,
        [ 0.        , -1.        ],
        [ 0.19611614, -0.98058068],
        [ 0.        ,  0.        ]],

       ...,

       [[ 0.        ,  0.        ],
        [ 0.70710678,  0.70710678],
        [ 0.70710678,  0.70710678],
        ...,
        [ 0.70710678, -0.70710678],
        [ 0.70710678, -0.70710678],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.70710678,  0.70